<a href="https://colab.research.google.com/github/shunsuketama/Shintama/blob/main/Plotly%E3%82%92%E7%94%A8%E3%81%84%E3%81%9FOCT%E3%81%AE3D%E5%86%8D%E6%A7%8B%E6%88%90_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Plotlyを用いたOCTの3D再構成**

In [ ]:
import os
import glob
import re
import numpy as np
import cv2
import plotly.graph_objects as go
from natsort import natsorted
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file_path = '/content/drive/MyDrive/Deep_learning/Shiraki_OCT_segmentation/Shiraki_OCT_dataset/Shiraki_OCT.zip'
destination_path = '/content'

# 目的のディレクトリが存在しない場合は作成
os.makedirs(destination_path, exist_ok=True)

# ファイルを解凍
!unzip -q "{zip_file_path}" -d "{destination_path}"
print(f"'{zip_file_path}' を '{destination_path}' に解凍しました。")

unzip:  cannot find or open /content/drive/MyDrive/Deep_learning/Shiraki_OCT_segmentation/Shiraki_OCT_dataset/Shiraki_OCT.zip, /content/drive/MyDrive/Deep_learning/Shiraki_OCT_segmentation/Shiraki_OCT_dataset/Shiraki_OCT.zip.zip or /content/drive/MyDrive/Deep_learning/Shiraki_OCT_segmentation/Shiraki_OCT_dataset/Shiraki_OCT.zip.ZIP.
'/content/drive/MyDrive/Deep_learning/Shiraki_OCT_segmentation/Shiraki_OCT_dataset/Shiraki_OCT.zip' を '/content' に解凍しました。


In [ ]:
target_dir = '/content/Shiraki_OCT/Original_images/11846715'

# 画像ファイルのリストを取得
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tif', '*.tiff']
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(target_dir, ext)))

# ハイフン前の数値を抽出する関数
def get_number_before_hyphen(filepath):
    filename = os.path.basename(filepath)
    try:
        # ハイフンで分割して最初の部分を数値に変換
        return int(filename.split('-')[0])
    except (ValueError, IndexError):
        # 変換できない場合やハイフンがない場合は -1 を返す
        return -1

# 抽出した数値に基づいてファイルをソート
sorted_files = sorted(image_files, key=get_number_before_hyphen)

print(f"Found {len(sorted_files)} images.")
if len(sorted_files) > 0:
    print("First 5 sorted files:")
    for f in sorted_files[:5]:
        print(os.path.basename(f))

Found 0 images.


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import cv2

# Colab用レンダラー設定
pio.renderers.default = "colab"

# 画像読み込み（前のコードと同じ）
images = []
for f in sorted_files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img_small = cv2.resize(img, (100, 100))  # さらに小さくして軽量化 (100x100ピクセルにリサイズ)
        images.append(img_small)

volume = np.array(images, dtype=np.float32)
print(f"Volume shape: {volume.shape}")  # (n_slices, height, width)

# 正しい座標グリッドの作成
n_slices, height, width = volume.shape
Z, Y, X = np.mgrid[0:n_slices, 0:height, 0:width]

# 値を正規化（0-255 → 0-1）
vol_normalized = volume / 255.0

# 閾値設定（背景を除外）
threshold = 0.25  # 調整してください

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vol_normalized.flatten(),
    isomin=threshold,
    isomax=0.9,
    opacity=0.8,
    surface_count=15,
    colorscale='Gray',
    caps=dict(x_show=False, y_show=False, z_show=False),
))

fig.update_layout(
    title="OCT 3D Volume",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z (Slice)",
        aspectmode='data',
    ),
    width=700,
    height=600,
)

fig.show()

Volume shape: (5, 100, 100)
